In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pennylane as qml

from sklearn.model_selection import train_test_split

# Direct classes included for easyness
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential

In [2]:
# Load data
np_data = np.load("data/dataRS.npy")

noise_output = np_data[:,-1].reshape(len(np_data),1)
input = np_data[:,:-1]

# split data
test_split = 0.33
x_train, x_test, y_train, y_test = train_test_split(input, noise_output, test_size=test_split)
# seed = np.random.seed(156703)
# x_train, x_test = tf.keras.utils.split_dataset(input, right_size=test_split, shuffle = True, seed=seed)
# y_train, y_test = tf.keras.utils.split_dataset(noise_output, right_size=test_split, shuffle = True, seed=seed)

In [10]:
print(np.random.rand(n_layers,n_qubits))
print(x_train[0,:])

[[0.54758558 0.54228646 0.67341617 0.30997225 0.06246545]
 [0.90296549 0.99427865 0.91982637 0.86783386 0.59014144]
 [0.50061688 0.00909228 0.22214854 0.73796146 0.82122782]
 [0.82764905 0.27674866 0.38866639 0.9743615  0.61135389]
 [0.03174203 0.24131792 0.85962804 0.14602323 0.05489669]
 [0.04023521 0.4863822  0.71978097 0.28301877 0.77353286]]
[0.11616162 0.24324324 0.45454545 0.         0.08367534]


In [16]:
from pennylane import numpy as np
from customEntanglerLayers import ToffoliEntanglerLayers 
# In a first try, we use as much qubits as we have input params

n_qubits = x_train.shape[1]
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}
n_qcParams = n_layers*n_qubits

dev = qml.device("default.qubit", wires=n_qubits)  # number of shots default?

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='X')
    #qml.StronglyEntanglingLayers(weight_shapes, wires=range(n_qubits), rotation=qml.RX)
    ToffoliEntanglerLayers(weights, wires=range(n_qubits), rotation=qml.RX)
    
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
# Visualize the quantum circuit
weights = np.random.rand(n_layers,n_qubits)
print(qml.draw(qlayer)(x_train,weights))
fig2, ax = qml.draw_mpl(qlayer)(weights)
plt.show()
fig2.savefig("archive/Toffoli-18032023.png")
fig2.savefig("archive/Toffoli-18032023.pdf")

TypeError: KerasLayer.call() takes 2 positional arguments but 3 were given

In [ ]:
#Create a simple ANN
clear_session()
layer_0 = Input(shape=(x_train.shape[1],))
layer_1 = Dense(5, activation="relu")
layer_2 = Dense(5, activation="linear")
ann = Sequential([layer_0, layer_1, qlayer, layer_2])
print(f"Params in the quantum layer to train: {n_qcParams}")
ann.summary()

plot_model(ann)

In [ ]:
#Training
#['mean_squared_error', 'mean_absolute_error','mean_squared_logarithmic_error', 'mean_absolute_percentage_error']
opt = tf.keras.optimizers.RMSprop(learning_rate=0.2)
ann.compile(opt, loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])
ann_history = ann.fit(x_train, y_train, epochs=50, batch_size=25, validation_split=0.25)

In [ ]:
import numpy as np
def plot_metrics(history):
    n = len(history.history.keys())//2
    fig,axs = plt.subplots(1,n, figsize=(18,5))

    for i,[key,val] in enumerate(history.history.items()):
        axs[i%n].plot(history.history[key], lw=4, label=key.replace("_", " "))
    
    for ax in axs:
        #ax.set_yscale("log")
        ax.set_xlabel("epoch", fontsize=16)
        ax.legend(fontsize=14)
plot_metrics(ann_history)
plt.savefig("archive/history-toffoli-16032023.png")

In [ ]:
evaluation = ann.evaluate(x_test[:-1], y_test[:-1])

In [ ]:
ann.save("archive/model-toffoli-16032023.h5")
config = ann.get_config()
np.save("archive/config-toffoli-16032023.npy", config)